In [1]:
from keras.models import Model, Input, Sequential
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda, Flatten
from keras.models import model_from_json
import numpy as np

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Using TensorFlow backend.


Loaded model from disk


In [2]:
with open("X_TEST.txt", "r") as xtest_file:
    x_test = xtest_file.read()
xtest_file.close() 

x_test = x_test.split(' ')
new_xTest = []
for x in x_test:
    if x != '':
        val = int(x)
        new_xTest.append(val)
        
x_test = new_xTest

X_TEST = []
temp = []
count = 1
for i in x_test:
    if count <= 30:
        temp.append(i)
    else:
        X_TEST.append(temp)
        temp = []
        temp.append(i)
        count = 1
    count = count + 1
        
with open("Y_TEST.txt", "r") as ytest_file:
    y_test = ytest_file.read()
ytest_file.close() 

y_test = y_test.split(' ')
new_yTest = []
for y in y_test:
    if y != '':
        val = float(y)
        new_yTest.append(val)
        
y_test = new_yTest

words = []
temp = []
count = 1    
for i in y_test:
    if count <= 25:
        temp.append(i)
    else:
        words.append(temp)
        temp = []
        temp.append(i)
        count = 1
    count = count + 1

sent = []
temp = []
count = 1    
for i in words:
    if count <= 30:
        temp.append(i)
    else:
        sent.append(temp)
        temp = []
        temp.append(i)
        count = 1
    count = count + 1

Y_TEST = sent

In [3]:
import json

with open("dictionary.json", "r") as dict_file:
    word2index = json.load(dict_file)
dict_file.close() 

In [4]:
with open("target.txt", "r") as target_file:
    targets = target_file.read()
target_file.close() 

targets = targets.split(', ')

In [18]:
pred = loaded_model.predict( [X_TEST] , verbose = 1)

976/976 [==============================] - 17s 17ms/step


In [25]:
Y_TEST = np.array(Y_TEST)
new_sent = []
for j in Y_TEST:
    for k in j:
        for l in range(len(k)):
            if(k[l] > 0):
                new_sent.append(l)
new_ytest = []
temp = []
count = 1
for i in new_sent:
    if count <= 30:
        temp.append(i)
    else:
        new_ytest.append(temp)
        temp = []
        temp.append(i)
        count = 1
    count = count + 1            
            
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(targets[p_i].replace("xxxPADDINGxxx", "O"))
        out.append(out_i)
    return out
def test2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(targets[p].replace("xxxPADDINGxxx", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(pred)
test_labels = test2label(new_ytest)

In [26]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

print(classification_report(test_labels, pred_labels))

                precision    recall  f1-score   support

         Genre       0.57      0.67      0.62       406
          Plot       0.32      0.38      0.35       813
Character_Name       0.33      0.31      0.32       146
  Relationship       0.32      0.09      0.15        85
          Year       0.82      0.88      0.85       336
       Opinion       0.00      0.00      0.00        97
        Origin       0.11      0.16      0.13        86
         Actor       0.77      0.83      0.80       621
      Director       0.68      0.70      0.69       206
         Quote       0.11      0.09      0.10        11
         Award       0.40      0.44      0.42        43
    Soundtrack       0.00      0.00      0.00         7

     micro avg       0.53      0.57      0.55      2857
     macro avg       0.52      0.57      0.54      2857



In [42]:
i = 30
p = loaded_model.predict( np.array([X_TEST[i]]) )
p = np.argmax(p, axis=-1)

print("{:14} {:20}: {}".format("Word", "True", "Pred"))
for w,true,pred in zip(X_TEST[i],new_ytest[i],p[0]):
    for w2i in word2index:
        if (w == w2i['index']):
            if w2i['word'] != "xxxPADDINGxxx":
                print("{:14} {:20}: {}".format(w2i['word'],targets[true],targets[pred]))

Word           True                : Pred
what           O                   : O
1941           B-Year              : B-Year
humphrey       B-Actor             : B-Actor
bogart         I-Actor             : I-Actor
film           O                   : O
concerned      O                   : O
the            B-Plot              : I-Plot
search         I-Plot              : I-Plot
for            I-Plot              : I-Plot
the            I-Plot              : I-Plot
stuff          I-Plot              : I-Plot
that           I-Plot              : I-Plot
dreams         I-Plot              : I-Plot
are            I-Plot              : I-Plot
made           I-Plot              : I-Plot
out            I-Plot              : I-Plot
of             I-Plot              : I-Plot
